In [0]:
%sql
SHOW TABLES IN samples.tpch;


database,tableName,isTemporary
tpch,customer,false
tpch,lineitem,false
tpch,nation,false
tpch,orders,false
tpch,part,false
tpch,partsupp,false
tpch,region,false
tpch,supplier,false


In [0]:
lineitem_df = spark.table("samples.tpch.lineitem")

lineitem_df.printSchema()
lineitem_df.show(5)
print("Row count:", lineitem_df.count())

root
 |-- l_orderkey: long (nullable = true)
 |-- l_partkey: long (nullable = true)
 |-- l_suppkey: long (nullable = true)
 |-- l_linenumber: integer (nullable = true)
 |-- l_quantity: decimal(18,2) (nullable = true)
 |-- l_extendedprice: decimal(18,2) (nullable = true)
 |-- l_discount: decimal(18,2) (nullable = true)
 |-- l_tax: decimal(18,2) (nullable = true)
 |-- l_returnflag: string (nullable = true)
 |-- l_linestatus: string (nullable = true)
 |-- l_shipdate: date (nullable = true)
 |-- l_commitdate: date (nullable = true)
 |-- l_receiptdate: date (nullable = true)
 |-- l_shipinstruct: string (nullable = true)
 |-- l_shipmode: string (nullable = true)
 |-- l_comment: string (nullable = true)

+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+----------------+----------+--------------------+
|l_orderkey|l_partkey|l_suppkey|l_linenumber|l_quantity|l_extendedprice|l_discount|l_tax|l

In [0]:
tpch_queries = {
    "Q1": """
        SELECT
          l_returnflag,
          l_linestatus,
          SUM(l_quantity)                                        AS sum_qty,
          SUM(l_extendedprice)                                   AS sum_base_price,
          SUM(l_extendedprice * (1 - l_discount))                AS sum_disc_price,
          SUM(l_extendedprice * (1 - l_discount) * (1 + l_tax))  AS sum_charge,
          AVG(l_quantity)                                        AS avg_qty,
          AVG(l_extendedprice)                                   AS avg_price,
          AVG(l_discount)                                        AS avg_disc,
          COUNT(*)                                               AS count_order
        FROM samples.tpch.lineitem
        WHERE l_shipdate <= DATE '1998-12-01' - INTERVAL 90 DAYS
        GROUP BY l_returnflag, l_linestatus
        ORDER BY l_returnflag, l_linestatus
    """,

    "Q2": """
        SELECT
          s.s_acctbal,
          s.s_name,
          n.n_name,
          p.p_partkey,
          p.p_mfgr,
          s.s_address,
          s.s_phone,
          s.s_comment
        FROM samples.tpch.part p
        JOIN samples.tpch.partsupp ps
          ON p.p_partkey = ps.ps_partkey
        JOIN samples.tpch.supplier s
          ON s.s_suppkey = ps.ps_suppkey
        JOIN samples.tpch.nation n
          ON s.s_nationkey = n.n_nationkey
        JOIN samples.tpch.region r
          ON n.n_regionkey = r.r_regionkey
        WHERE
          p.p_size = 15
          AND p.p_type LIKE '%BRASS'
          AND r.r_name = 'EUROPE'
          AND ps.ps_supplycost = (
            SELECT MIN(ps2.ps_supplycost)
            FROM samples.tpch.partsupp ps2
            JOIN samples.tpch.supplier s2
              ON s2.s_suppkey = ps2.ps_suppkey
            JOIN samples.tpch.nation n2
              ON s2.s_nationkey = n2.n_nationkey
            JOIN samples.tpch.region r2
              ON n2.n_regionkey = r2.r_regionkey
            WHERE
              ps2.ps_partkey = p.p_partkey
              AND r2.r_name = 'EUROPE'
          )
        ORDER BY s.s_acctbal DESC, n.n_name, s.s_name, p.p_partkey
        LIMIT 100
    """,

    "Q3": """
        SELECT
          o.o_orderkey,
          SUM(l.l_extendedprice * (1 - l.l_discount)) AS revenue,
          o.o_orderdate,
          o.o_shippriority
        FROM samples.tpch.customer c
        JOIN samples.tpch.orders o
          ON c.c_custkey = o.o_custkey
        JOIN samples.tpch.lineitem l
          ON l.l_orderkey = o.o_orderkey
        WHERE
          c.c_mktsegment = 'BUILDING'
          AND o.o_orderdate < DATE '1995-03-15'
          AND l.l_shipdate > DATE '1995-03-15'
        GROUP BY o.o_orderkey, o.o_orderdate, o.o_shippriority
        ORDER BY revenue DESC, o.o_orderdate
        LIMIT 10
    """,

    "Q4": """
        SELECT
          o.o_orderpriority,
          COUNT(*) AS order_count
        FROM samples.tpch.orders o
        WHERE
          o.o_orderdate >= DATE '1993-07-01'
          AND o.o_orderdate <  DATE '1993-10-01'
          AND EXISTS (
            SELECT 1
            FROM samples.tpch.lineitem l
            WHERE l.l_orderkey = o.o_orderkey
              AND l.l_commitdate < l.l_receiptdate
          )
        GROUP BY o.o_orderpriority
        ORDER BY o.o_orderpriority
    """,

    "Q5": """
        SELECT
          n.n_name,
          SUM(l.l_extendedprice * (1 - l.l_discount)) AS revenue
        FROM samples.tpch.customer c
        JOIN samples.tpch.orders o
          ON c.c_custkey = o.o_custkey
        JOIN samples.tpch.lineitem l
          ON l.l_orderkey = o.o_orderkey
        JOIN samples.tpch.supplier s
          ON s.s_suppkey = l.l_suppkey
        JOIN samples.tpch.nation n
          ON s.s_nationkey = n.n_nationkey
        JOIN samples.tpch.region r
          ON n.n_regionkey = r.r_regionkey
        WHERE
          r.r_name = 'ASIA'
          AND o.o_orderdate >= DATE '1994-01-01'
          AND o.o_orderdate <  DATE '1995-01-01'
        GROUP BY n.n_name
        ORDER BY revenue DESC
    """,

    "Q6": """
        SELECT
          SUM(l_extendedprice * l_discount) AS revenue
        FROM samples.tpch.lineitem
        WHERE
          l_shipdate >= DATE '1994-01-01'
          AND l_shipdate <  DATE '1995-01-01'
          AND l_discount BETWEEN 0.05 AND 0.07
          AND l_quantity < 24
    """,

    "Q7": """
        SELECT
          n1.n_name AS supp_nation,
          n2.n_name AS cust_nation,
          YEAR(l.l_shipdate) AS l_year,
          SUM(l.l_extendedprice * (1 - l.l_discount)) AS revenue
        FROM samples.tpch.supplier s
        JOIN samples.tpch.lineitem l
          ON s.s_suppkey = l.l_suppkey
        JOIN samples.tpch.orders o
          ON o.o_orderkey = l.l_orderkey
        JOIN samples.tpch.customer c
          ON c.c_custkey = o.o_custkey
        JOIN samples.tpch.nation n1
          ON s.s_nationkey = n1.n_nationkey
        JOIN samples.tpch.nation n2
          ON c.c_nationkey = n2.n_nationkey
        WHERE
          (
            (n1.n_name = 'FRANCE'  AND n2.n_name = 'GERMANY') OR
            (n1.n_name = 'GERMANY' AND n2.n_name = 'FRANCE')
          )
          AND l.l_shipdate >= DATE '1995-01-01'
          AND l.l_shipdate <= DATE '1996-12-31'
        GROUP BY n1.n_name, n2.n_name, YEAR(l.l_shipdate)
        ORDER BY supp_nation, cust_nation, l_year
    """,

    "Q8": """
        SELECT
          YEAR(o.o_orderdate) AS o_year,
          SUM(
            CASE
              WHEN n2.n_name = 'BRAZIL'
              THEN l.l_extendedprice * (1 - l.l_discount)
              ELSE 0
            END
          ) / SUM(l.l_extendedprice * (1 - l.l_discount)) AS mkt_share
        FROM samples.tpch.part p
        JOIN samples.tpch.lineitem l
          ON p.p_partkey = l.l_partkey
        JOIN samples.tpch.orders o
          ON o.o_orderkey = l.l_orderkey
        JOIN samples.tpch.customer c
          ON c.c_custkey = o.o_custkey
        JOIN samples.tpch.nation n1
          ON c.c_nationkey = n1.n_nationkey
        JOIN samples.tpch.region r
          ON n1.n_regionkey = r.r_regionkey
        JOIN samples.tpch.supplier s
          ON s.s_suppkey = l.l_suppkey
        JOIN samples.tpch.nation n2
          ON s.s_nationkey = n2.n_nationkey
        WHERE
          r.r_name = 'AMERICA'
          AND o.o_orderdate >= DATE '1995-01-01'
          AND o.o_orderdate <= DATE '1996-12-31'
          AND p.p_type = 'ECONOMY ANODIZED STEEL'
        GROUP BY YEAR(o.o_orderdate)
        ORDER BY o_year
    """,

    "Q9": """
        SELECT
          n.n_name,
          YEAR(o.o_orderdate) AS o_year,
          SUM(
            l.l_extendedprice * (1 - l.l_discount)
            - ps.ps_supplycost * l.l_quantity
          ) AS sum_profit
        FROM samples.tpch.lineitem l
        JOIN samples.tpch.orders o
          ON o.o_orderkey = l.l_orderkey
        JOIN samples.tpch.part p
          ON p.p_partkey = l.l_partkey
        JOIN samples.tpch.supplier s
          ON s.s_suppkey = l.l_suppkey
        JOIN samples.tpch.partsupp ps
          ON p.p_partkey = ps.ps_partkey
         AND s.s_suppkey = ps.ps_suppkey
        JOIN samples.tpch.nation n
          ON s.s_nationkey = n.n_nationkey
        WHERE
          p.p_name LIKE '%green%'
        GROUP BY n.n_name, YEAR(o.o_orderdate)
        ORDER BY n.n_name, o_year DESC
    """,

    "Q10": """
        SELECT
          c.c_custkey,
          c.c_name,
          SUM(l.l_extendedprice * (1 - l.l_discount)) AS revenue,
          c.c_acctbal,
          n.n_name,
          c.c_address,
          c.c_phone,
          c.c_comment
        FROM samples.tpch.customer c
        JOIN samples.tpch.orders o
          ON c.c_custkey = o.o_custkey
        JOIN samples.tpch.lineitem l
          ON l.l_orderkey = o.o_orderkey
        JOIN samples.tpch.nation n
          ON c.c_nationkey = n.n_nationkey
        WHERE
          o.o_orderdate >= DATE '1993-10-01'
          AND o.o_orderdate <  DATE '1994-01-01'
          AND l.l_returnflag = 'R'
        GROUP BY
          c.c_custkey,
          c.c_name,
          c.c_acctbal,
          n.n_name,
          c.c_address,
          c.c_phone,
          c.c_comment
        ORDER BY revenue DESC
        LIMIT 20
    """
}


We only considering cold run for this stage.    
Reason:  
1.The current SQL is performed on serverless compute
2.In real-word senario, most of the time people work on analysis with data market layer. Warm run is the most common way to do it. The main focus is warm steady-state throughput and latency

Procedure:  
we measure warm query runtimes by executing each query 3 times and reporting the minimum and average. This approximates steady-state analytics performance where data and execution paths are cached

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS bench;

CREATE TABLE IF NOT EXISTS bench.tpch_results (
  query       STRING,
  warm_min_ms DOUBLE,
  warm_avg_ms DOUBLE,
  run_ts      TIMESTAMP
);


In [0]:
import time
from datetime import datetime

def run_tpch_suite_warm_only(queries: dict, repeats: int = 3, log_table: str | None = "bench.tpch_results"):
    rows = []

    for name, sql_text in queries.items():
        warm_times = []

        for i in range(repeats):
            t0 = time.time()
            spark.sql(sql_text).count()   # force execution
            warm_times.append((time.time() - t0) * 1000)

        warm_min_ms = min(warm_times)
        warm_avg_ms = sum(warm_times) / len(warm_times)

        print(f"{name}: warm_min={warm_min_ms:.1f} ms, warm_avg={warm_avg_ms:.1f} ms over {repeats} runs")

        rows.append((name, warm_min_ms, warm_avg_ms, datetime.now()))

    result_df = spark.createDataFrame(
        rows,
        schema=["query",  "warm_min_ms", "warm_avg_ms", "run_ts"]
    )

    if log_table is not None:
        result_df.write.mode("append").saveAsTable(log_table)

    return result_df


In [0]:
results = run_tpch_suite_warm_only(tpch_queries, repeats=3)
results.show()

Q1: warm_min=805.7 ms, warm_avg=828.0 ms over 3 runs
Q2: warm_min=2120.6 ms, warm_avg=2380.0 ms over 3 runs
Q3: warm_min=2154.0 ms, warm_avg=2272.4 ms over 3 runs
Q4: warm_min=1306.9 ms, warm_avg=1392.2 ms over 3 runs
Q5: warm_min=2266.6 ms, warm_avg=2371.3 ms over 3 runs
Q6: warm_min=563.9 ms, warm_avg=604.7 ms over 3 runs
Q7: warm_min=2080.0 ms, warm_avg=2168.9 ms over 3 runs
Q8: warm_min=2374.5 ms, warm_avg=2521.8 ms over 3 runs
Q9: warm_min=2453.1 ms, warm_avg=2493.3 ms over 3 runs
Q10: warm_min=2218.0 ms, warm_avg=2268.8 ms over 3 runs
+-----+------------------+------------------+--------------------+
|query|       warm_min_ms|       warm_avg_ms|              run_ts|
+-----+------------------+------------------+--------------------+
|   Q1| 805.7315349578857| 827.9685179392496|2025-12-02 04:33:...|
|   Q2| 2120.558261871338| 2380.033016204834|2025-12-02 04:33:...|
|   Q3| 2154.038906097412|2272.3934650421143|2025-12-02 04:33:...|
|   Q4| 1306.947946548462|1392.1873569488525|2025-1